In [12]:
import duckdb


In [3]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

NameError: name 'duckdb' is not defined

In [18]:
df = con.execute("""
                 SELECT *
                 FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2026-02-13'
                 ) AS ranked
                 WHERE row = 1
                """).fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,1002,MARTELO,BT50,100,1500,z0019_1.csv,2026-02-13 16:48:29.181069,1
1,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-02-13 16:48:29.181069,1
2,1003,PREGO,BT10,100,150,z0019_1.csv,2026-02-13 16:48:29.181069,1
3,1005,MACHADO,BT50,100,1500,z0019_2.csv,2026-02-13 16:48:01.184980,1
4,1004,SERRA,BT50,100,100,z0019_2.csv,2026-02-13 16:48:01.184980,1


In [16]:
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,1004,SERRA,BT50,100,100,z0019_2.csv,2026-02-13 16:48:01.184980
1,1005,MACHADO,BT50,100,1500,z0019_2.csv,2026-02-13 16:48:01.184980
2,1003,PREGO,BT10,100,150,z0019_2.csv,2026-02-13 16:48:01.184980
3,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-02-13 16:48:29.181069
4,1002,MARTELO,BT50,100,1500,z0019_1.csv,2026-02-13 16:48:29.181069
5,1003,PREGO,BT10,100,150,z0019_1.csv,2026-02-13 16:48:29.181069


In [29]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={"NATBR": "id"})
df_final = df_final.rename(columns={"MAKTX": "nm_produto"})
df_final = df_final.rename(columns={"WERKS": "id_categoria"})
df_final = df_final.rename(columns={"MAINS": "id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "vl_preco"})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,1002,MARTELO,BT50,100,1500
1,1001,PARAFUSO,BT10,100,100
2,1003,PREGO,BT10,100,150
3,1005,MACHADO,BT50,100,1500
4,1004,SERRA,BT50,100,100


In [30]:
df2 = df_final
df_final.dtypes

id               str
nm_produto       str
id_categoria     str
id_fornecedor    str
vl_preco         str
dtype: object

In [36]:
df2 = df2.astype({
    'id': 'int32',
    'nm_produto': 'string',
    'id_categoria': 'string',
    'id_fornecedor': 'int32',
    'vl_preco': 'float64'
})
df2.dtypes

id                 int32
nm_produto        string
id_categoria      string
id_fornecedor      int32
vl_preco         float64
dtype: object

In [43]:
con.execute("""
                CREATE TABLE IF NOT EXISTS produtos (
                    id BIGINT,
                    nm_produto STRING,
                    id_categoria STRING,
                    id_fornecedor BIGINT,
                    vl_preco FLOAT
                )
            """).fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,1002,MARTELO,BT50,100,1500,z0019_1.csv,2026-02-13 16:48:29.181069,1
1,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-02-13 16:48:29.181069,1
2,1003,PREGO,BT10,100,150,z0019_1.csv,2026-02-13 16:48:29.181069,1
3,1005,MACHADO,BT50,100,1500,z0019_2.csv,2026-02-13 16:48:01.184980,1
4,1004,SERRA,BT50,100,100,z0019_2.csv,2026-02-13 16:48:01.184980,1


In [57]:
df2.head(10)
df_resultado = con.execute('''SELECT * FROM produtos ORDER BY id''').fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,1001,PARAFUSO,BT10,100,100.0
1,1002,MARTELO,BT50,100,1500.0
2,1003,PREGO,BT10,100,150.0
3,1004,SERRA,BT50,100,100.0
4,1005,MACHADO,BT50,100,1500.0


In [52]:
con.execute('''INSERT INTO produtos SELECT * FROM df2''').fetchdf()

,Count
0,5


In [2]:
con.close()

NameError: name 'con' is not defined